# CS 3110/3990: Data Privacy
## In-Class Exercise, Week of 9/23/2024

In [2]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('https://github.com/jnear/cs3110-data-privacy/raw/main/homework/adult_with_pii.csv')

## Question 1

For various values of $b$, write code to print out the percent error of summing the ages in the `adult` dataset, 
clipped to each value of $b$.

In [3]:
bs = range(1, 100, 10)
real_sum = adult['Age'].sum()

for b in bs:
    result = adult['Age'].clip(upper=b).sum()
    print(b, result)

1 32561
11 358171
21 678374
31 935798
41 1108069
51 1201865
61 1241700
71 1253315
81 1255772
91 1256257


What value of $b$ is the best?

Probably around b=90
- This is large enough that we are not introducing bias by clipping
- It is around the smallest possible number that avoids bias

So b=90 is the best choice for the bias/variance tradeoff

## Question 2

For various values of $b$, print the result of a *differentially private* sum of ages, clipped to each value of $b$. Use $\epsilon = 0.1$.

In [4]:
bs = range(1, 100, 10)
real_sum = adult['Age'].sum()

# total privacy cost must be multiplied by the number of loop iterations
# by sequential composition
for b in bs:
    result = adult['Age'].clip(upper=b).sum()
    # privacy cost of this is 0.1
    dp_result = laplace_mech(result, sensitivity=b, epsilon=0.1)
    print(b, dp_result)
# total privacy cost is .1 * 10 = 1, by sequential compsition

1 32559.702159143217
11 358026.15927400277
21 678230.5958057839
31 935622.9632648794
41 1108890.3631344305
51 1201787.4230650607
61 1241026.9916714148
71 1254408.7419572899
81 1258799.7601484396
91 1254947.0801606472


Which value of $b$ is the best now? Does it differ?

Probably a "b" of between 70 and 90 is good, because things seem to level off at that point.

It's hard to say for sure because of the noise added for differential privacy.

## Question 3

Write an algorithm to *automatically pick the clipping parameter* for the age column. Your algorithm should satisfy differential privacy.

In [5]:
def pick_b(epsilon):
    # find the plateau in noisy sums with different clipping parameters, as in q2
    # if result(b+1) - result(b) <= 0 then stop
    bs = range(1, 100, 10)
    epsilon_i = epsilon / len(bs)

    # total privacy cost must be multiplied by the number of loop interations
    # by sequential composition
    old_result = 0
    for b in bs:
        result = adult['Age'].clip(upper=b).sum()
        # privacy cost of this is 0.1
        dp_result = laplace_mech(result, sensitivity=b, epsilon=epsilon_i)
        if dp_result - old_result <= 0:
            return b
        old_result = dp_result
    # If I have reached the end and not detected a plateau, the best thing I can do
    # is return the last element of the list of candidates
    return bs[-1]

pick_b(1.0)

91

In [6]:
# TEST CASE for question 3

many_trials = np.mean([pick_b(1.0) for _ in range(100)])
assert many_trials > 70
assert many_trials < 500

## Question 4

What is the privacy cost of your algorithm, and why?

The algortihm satisfies differential privacy for the target epsilon
- Sensitivity: In each iteration of the loop, we call the laplace mech on the sum, clipped with the parameter `b`, so the sensitivity of the sum is `b`
- Composition: We use the laplace mech 10 time because `bs` has 10 elements. The total privacy cost is 10*.1 = 1, by sequential composition.
- Post-processing: We are returning an element of `bs`, which is a list that contains no private data

## Question 5

Write code to generate a *histogram* of education numbers in the `adult` dataset.

In [7]:
def education_hist():
    return adult['Education'].value_counts()
   
education_hist()

HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: Education, dtype: int64

In [8]:
# TEST CASE for question 5
h = education_hist()
assert h['HS-grad'] == 10501
assert h['12th'] == 433
assert h['Doctorate'] == 413

## Question 6

Write code to release a *differentially private histogram* of education numbers.

In [12]:
def dp_education_hist(epsilon):
    orig_hist = education_hist()
    # sensitivity of each bin of the histogram is 1, because each bin is a count
    return orig_hist.apply(lambda x: laplace_mech(x, sensitivity=1, epsilon=epsilon))

dp_education_hist(1.0)

HS-grad         10501.183765
Some-college     7291.084983
Bachelors        5354.951842
Masters          1725.500591
Assoc-voc        1380.198672
11th             1175.320540
Assoc-acdm       1066.986720
10th              933.851406
7th-8th           645.215640
Prof-school       576.260533
9th               512.456123
12th              432.655556
Doctorate         411.683428
5th-6th           334.123155
1st-4th           167.808713
Preschool          50.888022
Name: Education, dtype: float64

In [10]:
# TEST CASE for question 6
h = dp_education_hist(1.0)
assert abs(h['HS-grad'] - 10501) < 100
assert abs(h['Doctorate'] - 413) < 100

## Question 7

What is the total privacy cost of `dp_education_hist`, and why?

An argument based on sequential composition:
1. Sensitivity: We use a sensitivity of 1 for the laplace mechanism, because every use of it is on a counting query (since every bin of a histogram is a count)
2. Sequential composition: We use the laplace machanism k times, where k is the number of bins in the histogram. Therefore the total privacy cost is `k*epsilon` by sequential composition.
3. Post-processing: We are returning the noisy histogram, where all counts have noise added, which satisfies differential privacy by post-processing.

A better argument based on parallel composition:
1. Sensitivity: We use a sensitivity of 1 for the laplace mechanism, because every use of it is on a counting query (since every bin of a histogram is a count)
2. Parallel composition: We use the laplace mechanism k times, where k is the number of bins in the histogram. But each bin of the histogram does not overlap with any other bin. Therefore the total privacy cost is `epsilon` by parallel composition.
3. Post-processing: We are returning the noisy histogram, where all counts have noise added, which satisfies differential privacy by post-processing.

## Question 8

Write code to generate a differentially private *contingency table* for the Education and Sex columns of the `adult` dataset.

In [13]:
def dp_crosstab_education_sex(epsilon):
    return adult[['Education', 'Sex', 'Occupation']].value_counts()

dp_crosstab_education_sex(1.0)

Education     Sex     Occupation      
HS-grad       Male    Craft-repair        1820
Bachelors     Male    Exec-managerial     1037
HS-grad       Female  Adm-clerical         968
Bachelors     Male    Prof-specialty       916
Some-college  Female  Adm-clerical         903
                                          ... 
Assoc-acdm    Female  Protective-serv        1
1st-4th       Female  Sales                  1
9th           Male    Priv-house-serv        1
Doctorate     Male    Transport-moving       1
9th           Male    Tech-support           1
Length: 361, dtype: int64

In [ ]:
# TEST CASE for question 8
ct = dp_crosstab_education_sex(1.0)
assert abs(ct['Female']['10th'] - 295) < 100
assert abs(ct['Male']['10th'] - 638) < 100
assert abs(ct['Female']['Bachelors'] - 1619) < 100

assert abs(ct['Female']['10th'] - 295) > 0
assert abs(ct['Male']['10th'] - 638) > 0
assert abs(ct['Female']['Bachelors'] - 1619) > 0

## Question 9

Does parallel composition apply for the contingency table in question 1? Why or why not?

YOUR ANSWER HERE

## Question 10

Does the number of columns used in constructing the contingency table matter for privacy cost? Does it matter for accuracy?

YOUR ANSWER HERE

## Question 11

Implement the Gaussian mechanism for $(\epsilon, \delta)$-differential privacy.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST CASE

results = [gaussian_mech(len(adult[adult['Age'] > 50]), 1, 1.0, 10e-5) for _ in range(100)]
errors = [pct_error(len(adult[adult['Age'] > 50]), r) for r in results]
print('mean error:', np.mean(errors))

assert np.mean(errors) > 0
assert np.mean(errors) < 2

## Question 12

How do the Laplace and Gaussian mechanisms compare in terms of relative error on the query "how many individuals are over 50 years old" with $\epsilon = 1$ and $\delta = 10^{-5}$?

In [ ]:
true_answer = len(adult[adult['Age'] > 50])

laplace_answers = [laplace_mech(true_answer, 1, 1) for _ in range(200)]
gaussian_answers = [gaussian_mech(true_answer, 1, 1, 10e-5) for _ in range(200)]

laplace_error = [pct_error(true_answer, a) for a in laplace_answers]
gaussian_error = [pct_error(true_answer, a) for a in gaussian_answers]

_, bins, _ = plt.hist(gaussian_error, bins=20, label='Gaussian')
plt.hist(laplace_error, bins=bins, label='Laplace', alpha=0.5)
plt.legend();

YOUR ANSWER HERE

## Not a Question - Just for reference

[Reference](https://uvm-plaid.github.io/programming-dp/notebooks/ch6.html#the-gaussian-mechanism)

In [ ]:
epsilon = 1
sensitivity = 1
delta = 1e-5
sigma_squared = 2 * sensitivity**2 * np.log(1.25 / delta) / (epsilon**2)
sigma = np.sqrt(sigma_squared)

def gauss_pdf(x):
    return 1/(sigma*np.sqrt(2*np.pi)) * np.exp(-(1/2)*(x/sigma)**2)

xs = np.linspace(-50, 50, 200)
ys1 = [gauss_pdf(x) for x in xs]
ys2 = [gauss_pdf(x+1) for x in xs]

plt.plot(xs,ys1)
plt.plot(xs,ys2)

# ratio < e^epsilon should hold
print('e^epsilon =', np.exp(epsilon))
ratios = [(x, y1 / y2) for x, y1, y2 in zip(xs, ys1, ys2)]
#ratios

In [ ]:
def laplace_pdf(x):
    return (1/2)*epsilon * np.exp(-np.abs(x)*epsilon)

xs = np.linspace(-50, 50, 200)
ys1 = [laplace_pdf(x) for x in xs]
ys2 = [laplace_pdf(x+1) for x in xs]

plt.plot(xs,ys1)
plt.plot(xs,ys2)

# ratio < e^epsilon should hold
print('e^epsilon =', np.exp(epsilon))
ratios = [(x, y1 / y2) for x, y1, y2 in zip(xs, ys1, ys2)]
#ratios